In [1]:
!pip install -q transformers accelerate sentence-transformers faiss-cpu pandas bitsandbytes


In [ ]:
import pandas as pd

csv_path = "/content/indian_startup_funding_2020_2025_sample.csv"  # change if needed
df = pd.read_csv(csv_path)

df.head()


,Startup,Industry,SubVertical,City,Investors,InvestmentType,InvestmentAmount_USD,Date
0,Housejoy,EdTech,K12,Mumbai,Lightspeed India,Seed,199000,19-04-2023
1,Groww,Media,Streaming,Bengaluru,IFC,Seed,1668000,28-01-2025
2,Groww,Mobility,Ride Sharing,Hyderabad,"Nexus Venture Partners, Peak XV",Series B,38052000,14-03-2021
3,FarmBox,Consumer Electronics,Wearables,Gurugram,"Kalaari Capital, Y Combinator",Seed,455000,11-09-2023
4,Udaan,RealEstate,Rental Tech,Mumbai,Bessemer Venture Partners,Seed,89000,31-01-2024


In [ ]:
df = df.dropna(subset=["Startup", "InvestmentType", "InvestmentAmount_USD"])
df = df.fillna("Unknown")
df.rename(columns={'Startup': 'Startup Name', 'InvestmentType': 'Funding Round', 'InvestmentAmount_USD': 'Amount in USD'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Year'] = df['Date'].dt.year
df.columns

Index(['Startup Name', 'Industry', 'SubVertical', 'City', 'Investors',
       'Funding Round', 'Amount in USD', 'Date', 'Year'],
      dtype='object')

In [ ]:
def funding_row_to_text(row):
    return (
        f"The startup {row['Startup Name']} raised {row['Amount in USD']} USD "
        f"in a {row['Funding Round']} round in {row['Year']} "
        f"from investors {row['Investors']} "
        f"and is based in {row['City']} operating in the {row['Industry']} sector."
    )

documents = df.apply(funding_row_to_text, axis=1).tolist()

print(documents[0])


The startup Housejoy raised 199000 USD in a Seed round in 2023 from investors Lightspeed India and is based in Mumbai operating in the EdTech sector.


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

embeddings = embedding_model.encode(documents, show_progress_bar=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Funding records indexed:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Funding records indexed: 1100


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def retrieve_context(query, k=5):
    query_embedding = embedding_model.encode([query])
    _, indices = index.search(query_embedding, k)
    return "\n".join([documents[i] for i in indices[0]])


In [ ]:
def funding_intelligence_copilot(query):
    context = retrieve_context(query)

    prompt = f"""
You are a startup funding intelligence AI for Indian founders.

RULES:
- Answer ONLY using the provided context
- Do NOT guess or add information
- If the answer is not found, say:
  "I do not have enough funding data to answer this."

CONTEXT:
{context}

QUESTION:
{query}

ANSWER:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

    output = llm.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.1,
        do_sample=False
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
print(funding_intelligence_copilot(
    "How much funding did fintech startups in Bangalore raise?"
))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Answer ONLY using the provided context
- Do NOT guess or add information
- If the answer is not found, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup FinFoods_246 raised 3188000 USD in a Series A round in 2024 from investors Blume Ventures and is based in Ahmedabad operating in the FoodTech sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is based in Chennai operating in the EdTech sector.
The startup QuantumAnalytics_589 raised 74657000 USD in a Growth round in 2022 from investors Nexus Venture Partners, Peak XV and is based in Pune operating in the FinTech sector.
The startup Freshworks raised 14717000 USD in a Serie

In [ ]:
print(funding_intelligence_copilot(
    "बैंगलोर में फिनटेक स्टार्टअप्स ने कितनी फंडिंग उठाई?"
))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Answer ONLY using the provided context
- Do NOT guess or add information
- If the answer is not found, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup CoreConnect_1085 raised 18272000 USD in a Series B round in 2024 from investors Matrix Partners India and is based in Chennai operating in the EdTech sector.
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is based in Chennai operating in the EdTech sector.
The startup SkillSolutions raised 8795000 USD in a Series A round in 2022 from investors Info Edge, Peak XV, SoftBank Vision Fund and is based in Bengaluru operating in the Logistics sector.
The startup CoreConnect raised 147

In [ ]:
print(funding_intelligence_copilot(
    "How do I apply for an IPO in India?"
))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Answer ONLY using the provided context
- Do NOT guess or add information
- If the answer is not found, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Delhivery raised 52000 USD in a Seed round in 2020 from investors Bessemer Venture Partners, Omidyar Network and is based in Pune operating in the Consumer Electronics sector.
The startup AutoFlow raised 6610000 USD in a Series A round in 2025 from investors Bessemer Venture Partners, Sequoia Capital India, Zodius Capital and is based in Gurugram operating in the HealthTech sector.
The startup AgriSense raised 1746000 USD in a Seed round in 2024 from investors Sequoia Capital India and is based in Chennai operating in the Retail sector.
The startup AgriFit raised 13365000 USD in a Series B round in 2020 from investors Sequoia Capital India and is based in Mumbai operating in the Mobility sector.
The startup AgriFit raised 1218000 US

In [ ]:
query = "fintech startups in Bangalore"
context = retrieve_context(query)

print("RETRIEVED CONTEXT:\n")
print(context)


RETRIEVED CONTEXT:

The startup WorkStack raised 1593000 USD in a Seed round in 2020 from investors Bessemer Venture Partners, Blume Ventures and is based in Bengaluru operating in the Retail sector.
The startup CoreConnect raised 14710000 USD in a Series B round in 2021 from investors LCatterton and is based in Bengaluru operating in the RealEstate sector.
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup QuantumSolutions raised 1272000 USD in a Seed round in 2024 from investors Sequoia Capital India and is based in Bengaluru operating in the AgriTech sector.
The startup FreshTech raised 9715000 USD in a Series B round in 2024 from investors Accel, LCatterton, Tiger Global Management and is based in Delhi operating in the RealEstate sector.


In [ ]:
print(funding_intelligence_copilot(
    "Which fintech startups in Bangalore raised funding?"
))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Answer ONLY using the provided context
- Do NOT guess or add information
- If the answer is not found, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is based in Chennai operating in the EdTech sector.
The startup QuantumAnalytics raised 73000 USD in a Seed round in 2024 from investors LCatterton and is based in Bengaluru operating in the RealEstate sector.
The startup QuantumSolutions raised 1272000 USD in a Seed round in 2024 from investors Sequoia Capital India and is based in Bengaluru operating in the AgriTech sector.
The startup FinFoods_246 raised 3188000 USD in a Series A round in 2

In [ ]:
def funding_intelligence_copilot(query):
    context = retrieve_context(query)

    prompt = f"""
You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
{context}

QUESTION:
{query}

ANSWER:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

    output = llm.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=False
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
print(funding_intelligence_copilot(
    "Which startups in Bangalore raised Series A funding?"
))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup SkillSolutions raised 8795000 USD in a Series A round in 2022 from investors Info Edge, Peak XV, SoftBank Vision Fund and is based in Bengaluru operating in the Logistics sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is based in Chennai operating in the EdTech sector.
The startup HomeAnalytics raised 1458000 USD in a Series A round in 2021 from investors Bessemer Venture Partners, Falcon Edge and is based in Bengaluru 

In [ ]:
print(funding_intelligence_copilot(
    "बैंगलोर में कौन से स्टार्टअप्स ने फंडिंग उठाई?"
))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup Delhivery raised 1418000 USD in a Series A round in 2021 from investors Accel, Mirae Asset, SoftBank Vision Fund and is based in Chennai operating in the E-commerce sector.
The startup WorkFit raised 1883000 USD in a Series A round in 2020 from investors Kedaara Capital, Tiger Global Management, Ventures India and is based in Delhi operating in the RealEstate sector.
The startup CoreConnect_1085 raised 18272000 USD in a Series B round in 2024 from investors Matrix Partners India and is based in Chennai o

In [ ]:
questions = [
    # English
    "Which startups raised funding in Bangalore?",
    "Which startups raised Series A funding?",
    "Which fintech startups raised funding?",
    "Which startups are based in Mumbai and raised funding?",
    "Which startups raised funding in 2021?",
    "Which startups raised Seed funding?",
    "Which startups received funding from Sequoia Capital?",
    "Which startups operate in the SaaS sector?",
    "Which startups raised Series B funding?",
    "Which startups raised funding in Delhi NCR?",

    # Hindi
    "बैंगलोर में किन स्टार्टअप्स ने फंडिंग उठाई?",
    "कौन से फिनटेक स्टार्टअप्स को फंडिंग मिली?",
    "2022 में किन स्टार्टअप्स को फंडिंग मिली?",
    "किस स्टार्टअप ने Series A फंडिंग उठाई?",

    # Tamil
    "பெங்களூரில் எந்த ஸ்டார்ட்அப்கள் நிதி பெற்றுள்ளன?",
    "எந்த ஃபின்டெக் ஸ்டார்ட்அப்களுக்கு நிதி கிடைத்தது?",
    "2021-ல் எந்த ஸ்டார்ட்அப்கள் நிதி பெற்றன?",

    # Telugu
    "బెంగళూరులో ఏ స్టార్టప్‌లు ఫండింగ్ పొందాయి?",
    "ఏ ఫిన్‌టెక్ స్టార్టప్‌లు పెట్టుబడులు పొందాయి?",
    "2023లో ఏ స్టార్టప్‌లు ఫండింగ్ పొందాయి?",

    # Validation (should refuse)
    "How do I IPO my startup in India?",
    "Which VC will definitely fund my startup?"
]


In [ ]:
for i, q in enumerate(questions, start=1):
    print("=" * 80)
    print(f"Q{i}: {q}\n")

    answer = funding_intelligence_copilot(q)

    print("Answer:")
    print(answer)
    print("\n")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q1: Which startups raised funding in Bangalore?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:

You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Freshworks raised 1654000 USD in a Seed round in 2025 from investors Matrix Partners India, Prosus Ventures and is based in Bengaluru operating in the RealEstate sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is based in Chennai operating in the EdTech sector.
The startup SkillSolutions raised 8795000 USD in a Series A round in 2022 from investors Info Edge, Peak XV, SoftBank Vision Fund and is based in Bengaluru operating in the Logistics sector.
The startup Delhivery raised 1418000 USD in a Series A round in 2021 from investors Accel, Mirae Asset, SoftBank Vision Fund and is based in Chen

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Answer:

You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup PharmEasy raised 24655000 USD in a Series B round in 2021 from investors LCatterton, Zodius Capital and is based in Gurugram operating in the HealthTech sector.
The startup SkillSolutions raised 16426000 USD in a Series B round in 2020 from investors Omidyar Network, Sequoia Capital India, Y Combinator and is based in Chennai operating in the Consumer Electronics sector.
The startup SkillSolutions raised 46994000 USD in a Series B round in 2023 from investors Bessemer Venture Partners, Omidyar Network, Sequoia Capital India and is based in Mumbai operating in the E-commerce sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investo

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:

You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup NextFlow raised 67565000 USD in a Series B round in 2021 from investors A91 Partners, Kedaara Capital, LCatterton and is based in Noida operating in the FoodTech sector.
The startup PharmEasy raised 24655000 USD in a Series B round in 2021 from investors LCatterton, Zodius Capital and is based in Gurugram operating in the HealthTech sector.
The startup SkillSolutions raised 16426000 USD in a Series B round in 2020 from investors Omidyar Network, Sequoia Capital India, Y Combinator and is based in Chennai operating in the Consumer Electronics sector.
The startup QuantumSolutions_441 raised 289340000 USD in a Series C round in 2020 from investors A91 Partners, Info Edge and is

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:

You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
The startup Delhivery raised 52000 USD in a Seed round in 2020 from investors Bessemer Venture Partners, Omidyar Network and is based in Pune operating in the Consumer Electronics sector.
The startup Delhivery raised 778000 USD in a Seed round in 2023 from investors Kalaari Capital and is based in Kolkata operating in the RealEstate sector.
The startup AutoFlow raised 6610000 USD in a Series A round in 2025 from investors Bessemer Venture Partners, Sequoia Capital India, Zodius Capital and is based in Gurugram operating in the HealthTech sector.
The startup NextSense_10 raised 54000 USD in a Seed round in 2020 from investors IFC, Kalaari Capital, Peak XV and is based in Delhi operating 

In [ ]:
results = []

for q in questions:
    results.append({
        "question": q,
        "answer": funding_intelligence_copilot(q)
    })

import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv("llm_outputs.csv", index=False)

print("Saved results to llm_outputs.csv")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Saved results to llm_outputs.csv


In [ ]:
def funding_intelligence_copilot(query):
    context = retrieve_context(query)

    prompt = f"""
You are a startup funding intelligence AI for Indian founders.

RULES:
- Use ONLY the information in the context
- You MAY summarize or rephrase
- Do NOT invent new facts
- If the context does not contain relevant information, say:
  "I do not have enough funding data to answer this."

CONTEXT:
{context}

QUESTION:
{query}

ANSWER:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

    output = llm.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.2,
        do_sample=False
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # ✅ KEEP ONLY THE ANSWER
    if "ANSWER:" in decoded:
        return decoded.split("ANSWER:")[-1].strip()
    else:
        return decoded.strip()


In [ ]:
questions = [
    # English
    "Which startups raised funding in Bangalore?",
    "Which startups raised Series A funding?",
    "Which fintech startups raised funding?",
    "Which startups are based in Mumbai and raised funding?",
    "Which startups raised funding in 2021?",
    "Which startups raised Seed funding?",
    "Which startups received funding from Sequoia Capital?",
    "Which startups operate in the SaaS sector?",
    "Which startups raised Series B funding?",
    "Which startups raised funding in Delhi NCR?",

    # Hindi
    "बैंगलोर में किन स्टार्टअप्स ने फंडिंग उठाई?",
    "कौन से फिनटेक स्टार्टअप्स को फंडिंग मिली?",
    "2022 में किन स्टार्टअप्स को फंडिंग मिली?",
    "किस स्टार्टअप ने Series A फंडिंग उठाई?",

    # Tamil
    "பெங்களூரில் எந்த ஸ்டார்ட்அப்கள் நிதி பெற்றுள்ளன?",
    "எந்த ஃபின்டெக் ஸ்டார்ட்அப்களுக்கு நிதி கிடைத்தது?",
    "2021-ல் எந்த ஸ்டார்ட்அப்கள் நிதி பெற்றன?",

    # Telugu
    "బెంగళూరులో ఏ స్టార్టప్‌లు ఫండింగ్ పొందాయి?",
    "ఏ ఫిన్‌టెక్ స్టార్టప్‌లు పెట్టుబడులు పొందాయి?",
    "2023లో ఏ స్టార్టప్‌లు ఫండింగ్ పొందాయి?",

    # Validation (should refuse)
    "How do I IPO my startup in India?",
    "Which VC will definitely fund my startup?"
]


In [ ]:
for i, q in enumerate(questions, start=1):
    print("=" * 80)
    print(f"Q{i}: {q}\n")

    answer = funding_intelligence_copilot(q)

    print("Answer:")
    print(answer)
    print("\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q1: Which startups raised funding in Bangalore?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:
Freshworks raised 1654000 USD in a Seed round in 2025, SkillSolutions raised 8795000 USD in a Series A round in 2022, and QuantumSolutions raised 1272000 USD in a Seed round in 2024. All these startups are based in Bangalore.


Q2: Which startups raised Series A funding?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Answer:
The following startups have raised funding: PharmEasy (24655000 USD in Series B), SkillSolutions (16426000 USD in Series B and 46994000 USD in Series B), QuantumSolutions_441 (289340000 USD in Series C), and ShareChat (4680000 USD in Series A). These startups are based in Gurugram, Mumbai, Mumbai, and Ahmedabad respectively, and they operate in the HealthTech, E-commerce, E-commerce, and AgriTech sectors.


Q20: 2023లో ఏ స్టార్టప్‌లు ఫండింగ్ పొందాయి?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:
The startup SkillSolutions raised 46994000 USD in 2023.


Q21: How do I IPO my startup in India?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:
I do not have enough funding data to answer this question based on the provided context. The context only mentions the funding rounds and investors of various startups, but it does not provide information on the specific requirements or process for IPOing a startup in India.


Q22: Which VC will definitely fund my startup?

Answer:
Based on the provided context, none of the investors have been identified as funding a startup in the same sector, location, or round more than once. Therefore, the context does not provide enough information to determine which VC will definitely fund your startup.




In [ ]:
results = []

for q in questions:
    results.append({
        "question": q,
        "answer": funding_intelligence_copilot(q)
    })

import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv("llm_outputs.csv", index=False)

print("Saved results to llm_outputs.csv")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Saved results to llm_outputs.csv
